In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("M11_Data.csv")
df.head()

,Month,Category,Caltex,Gulf,Mobil
0,Open,Engine Oil,140 : 000,199 : 000,141 : 000
1,NaN,GearBox Oil,198 : 000,132 : 000,121 : 000
2,Jan,Engine Oil,170 : 103,194 : 132,109 : 127
3,NaN,GearBox Oil,132 : 106,125 : 105,191 : 100
4,Feb,Engine Oil,112 : 133,138 : 113,171 : 101


# Part 1: Tidying and Reshaping Data

## 1.1: Convert dataframe from Wide to Long format.

In [3]:
def OilPurchased(x):
    x = x.split(":")
    return x[0].strip()
def OilConsumed(x):
    x = x.split(":")
    return x[1].strip()

df['Caltex_Purchased'] = df.Caltex.apply(OilPurchased)
df['Caltex_Consumed'] = df.Caltex.apply(OilConsumed)

df['Gulf_Purchased'] = df.Gulf.apply(OilPurchased)
df['Gulf_Consumed'] = df.Gulf.apply(OilConsumed)

df['Mobil_Purchased'] = df.Mobil.apply(OilPurchased)
df['Mobil_Consumed'] = df.Mobil.apply(OilConsumed)

df.drop(['Caltex', "Gulf", "Mobil"], inplace=True, axis=1)
df.head()

,Month,Category,Caltex_Purchased,Caltex_Consumed,Gulf_Purchased,Gulf_Consumed,Mobil_Purchased,Mobil_Consumed
0,Open,Engine Oil,140,000,199,000,141,000
1,NaN,GearBox Oil,198,000,132,000,121,000
2,Jan,Engine Oil,170,103,194,132,109,127
3,NaN,GearBox Oil,132,106,125,105,191,100
4,Feb,Engine Oil,112,133,138,113,171,101


In [4]:
longdf = df.melt(id_vars=['Month', 'Category'], var_name='Oil_Type')
longdf.value = pd.to_numeric(longdf.value)
longdf.head()

,Month,Category,Oil_Type,value
0,Open,Engine Oil,Caltex_Purchased,140
1,NaN,GearBox Oil,Caltex_Purchased,198
2,Jan,Engine Oil,Caltex_Purchased,170
3,NaN,GearBox Oil,Caltex_Purchased,132
4,Feb,Engine Oil,Caltex_Purchased,112


### 1.2.1: What was the amount of oil remaining for each type/brand at the end of the chronological period?

In [5]:
purchased = 0
consumed = 0
for ot in longdf.Oil_Type:
    if "Purchased" in ot:
        purchased += longdf[longdf.Oil_Type == ot].value.sum()
    else:
        consumed += longdf[longdf.Oil_Type == ot].value.sum()
longdf[longdf.Oil_Type == "Caltex_Purchased"].value.sum() - longdf[longdf.Oil_Type == "Caltex_Consumed"].value.sum()
print("Remaining Oil at the end of Chronological period is equal to", purchased - consumed)

Remaining Oil at the end of Chronological period is equal to 28980


### 1.2.2: What was the most consumed brand of oil across the two separate categories/types of oil?

In [6]:
catoilgrp = pd.DataFrame(longdf.groupby(["Category", 'Oil_Type'])['value'].sum())
print("Engine Oil cconsumed", catoilgrp[:6:2].sum())
print("GearBox Oil consumed", catoilgrp[6::2].sum())

Engine Oil cconsumed value    2191
dtype: int64
GearBox Oil consumed value    2200
dtype: int64


### 1.3: Converting dataframe from Long to Wide format.

In [46]:
widedf = pd.pivot_table(longdf, index="Month", columns=['Category', "Oil_Type"], values='value')
widedf

Category      Engine Oil                                                \
Oil_Type Caltex_Consumed Caltex_Purchased Gulf_Consumed Gulf_Purchased   
Month                                                                    
Apr                  150              149           118            117   
Feb                  133              112           113            138   
Jan                  103              170           132            194   
Jun                  129              159           138            170   
Mar                  100              184           141            141   
May                  139              170           119            104   
Open                   0              140             0            199   

Category                                 
Oil_Type Mobil_Consumed Mobil_Purchased  
Month                                    
Apr                 118             117  
Feb                 101             171  
Jan                 127             109  
Jun                 105             169  
Mar                 108             114  
May                 117             200  
Open                  0             141

# Part 2: Using Your GroupBy and Data Aggregation Skills

### Reading the Dataset from Github and renaming the columns.

In [9]:
bridges = pd.read_csv("https://raw.githubusercontent.com/AliHaider20/AIM-5001/main/Data%20Engineering/bridges.data.version1.csv", usecols=None)
bridges.columns  = ["Id", "River", "Location", "Erected", "Purpose",
                   "Length", "Lanes", "Clear_g", "T_or_D", "Material",
                   "Span", "Rel_L", "Type"]

bridges.head()

,Id,River,Location,Erected,Purpose,Length,Lanes,Clear_g,T_or_D,Material,Span,Rel_L,Type
0,E2,A,25,1819,HIGHWAY,1037,2,N,THROUGH,WOOD,SHORT,S,WOOD
1,E3,A,39,1829,AQUEDUCT,?,1,N,THROUGH,WOOD,?,S,WOOD
2,E5,A,29,1837,HIGHWAY,1000,2,N,THROUGH,WOOD,SHORT,S,WOOD
3,E6,M,23,1838,HIGHWAY,?,2,N,THROUGH,WOOD,?,S,WOOD
4,E7,A,27,1840,HIGHWAY,990,2,N,THROUGH,WOOD,MEDIUM,S,WOOD


### Grouping River, Purpose, and Material and getting the count.

In [10]:
count = pd.DataFrame(bridges.groupby(['River', 'Purpose', 'Material']).size())
count.columns = ["How many ?"]
count.head()

How many ?
River Purpose  Material            
A     AQUEDUCT IRON               1
               WOOD               3
      HIGHWAY  ?                  1
               IRON               2
               STEEL             21

### Grouping the Purpose, and Material with Length and getting the count. Creating a new df where Length values are not null.

In [11]:
bridges.replace("?", np.NaN, inplace=True)
NotnaLength = bridges[bridges.Length.notnull()]
NotnaLengthdf =  pd.DataFrame(NotnaLength.groupby(['Purpose', 'Material'])['Length'].count())
NotnaLength

,Id,River,Location,Erected,Purpose,Length,Lanes,Clear_g,T_or_D,Material,Span,Rel_L,Type
0,E2,A,25,1819,HIGHWAY,1037,2,N,THROUGH,WOOD,SHORT,S,WOOD
2,E5,A,29,1837,HIGHWAY,1000,2,N,THROUGH,WOOD,SHORT,S,WOOD
4,E7,A,27,1840,HIGHWAY,990,2,N,THROUGH,WOOD,MEDIUM,S,WOOD
5,E8,A,28,1844,AQUEDUCT,1000,1,N,THROUGH,IRON,SHORT,S,SUSPEN
6,E9,M,3,1846,HIGHWAY,1500,2,N,THROUGH,IRON,SHORT,S,SUSPEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,E86,A,33,1961,HIGHWAY,980,4,G,DECK,STEEL,MEDIUM,S-F,CONT-T
101,E85,M,9,1962,HIGHWAY,2213,4,G,DECK,STEEL,LONG,F,CONT-T
102,E84,A,24,1969,HIGHWAY,870,6,G,THROUGH,STEEL,MEDIUM,F,ARCH
103,E91,O,44,1975,HIGHWAY,3756,6,G,THROUGH,STEEL,LONG,F,ARCH


#### Created a function to fill Length value.

In [12]:
def fillLength(col1val, col2val):
    mean = round(NotnaLength[(NotnaLength["Purpose"] == col1val) & (NotnaLength["Material"] == col2val)].Length.astype(int).mean(), 2)
    tdf = bridges[(bridges["Purpose"] == col1val) & (bridges["Material"] == col2val)]
    ind = tdf[tdf.Length.isna() == True].index
    bridges.loc[ind, 'Length'] = str(mean)

- Replacing ? with NaN values to ease processing.
- Using the fillLength to fill the nan values.

In [13]:
for (purpose,mat) in NotnaLengthdf.index:
    fillLength(purpose, mat)
bridges.Length.fillna(bridges[bridges.Length.notnull()].Length.astype(float).mean(), inplace=True)
bridges.Length = bridges.Length.astype(float)

In [14]:
bridges[bridges.Length.isna()] # Check

,Id,River,Location,Erected,Purpose,Length,Lanes,Clear_g,T_or_D,Material,Span,Rel_L,Type


In [15]:
avgLendf = pd.DataFrame(bridges.groupby(['Purpose', 'Material'])['Length'].mean())
avgLendf.columns = ['Average Length']
avgLendf

Average Length
Purpose  Material                
AQUEDUCT IRON         1000.000000
         WOOD         1092.000000
HIGHWAY  IRON         1216.668571
         STEEL        1557.803922
         WOOD         1053.376000
RR       IRON         1100.000000
         STEEL        1946.850000
         WOOD         1555.288137
WALK     STEEL        1555.288137

In [16]:
ereclength = pd.DataFrame()
Erecbins = pd.cut(bridges.Erected, [1818, 1860, 1902, 1944, 1986])
bridges.groupby(Erecbins).Length.agg(["mean", "size", 'max', 'min'])

,mean,size,max,min
Erected,,,,
"(1818, 1860]",1154.166877,14,1555.288137,990.0
"(1860, 1902]",1590.314722,36,4558.000000,1000.0
"(1902, 1944]",1677.053837,36,3000.000000,860.0
"(1944, 1986]",1553.915537,21,3756.000000,804.0
